## Fully Connected layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
t_max = 2**8
dims = (1,15,15)
n_outputs = 10

np.random.seed(seed=39)
weights = (np.random.rand(n_outputs,np.product(dims)) - 0.5) / 5
biases = (np.random.rand(n_outputs) - 0.5) / 2

loihi_model = quartz.Network([
    layers.InputLayer(dims=dims, weight_acc=256),
    layers.Dense(weights=weights, biases=biases, weight_acc=256),
    layers.MonitorLayer(weight_acc=256),
])

# calc_probe = quartz.probe(loihi_model.layers[1].blocks[-3].neurons[0])
# sync_probe = quartz.probe(loihi_model.layers[1].blocks[-3].neurons[2])
# input_probe = quartz.probe(loihi_model.layers[0].blocks[0])
# hidden_probe9 = quartz.probe(loihi_model.layers[1].blocks[-3])
hidden1 = quartz.probe(loihi_model.layers[1].blocks[3])
# output_probe = quartz.probe(loihi_model.layers[2].blocks[0])
# bias9_block = loihi_model.layers[1].blocks[-2]
# bias9_probe = quartz.probe(bias9_block)
# bias7_block = loihi_model.layers[1].blocks[-8]
# bias7_probe = quartz.probe(bias7_block)

values = np.random.rand(np.product(dims)) # np.ones((np.product(dims))) * 0.5 np.zeros((np.product(dims))) # 
inputs = quartz.decode_values_into_spike_input(values, t_max)

pt_model = nn.Sequential(
    nn.Linear(in_features=np.product(dims), out_features=n_outputs), 
    nn.ReLU()
)
pt_model[0].weight = torch.nn.Parameter(torch.tensor(weights))
pt_model[0].bias = torch.nn.Parameter(torch.tensor(biases))
model_output = pt_model(torch.tensor(values)).detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
biases

In [ ]:
hidden1.plot()

## Convolutional Layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

t_max = 2**9
input_dims = (2,5,5)
weight_dims = (1,2,2,2)

kernel_size = weight_dims[2:]
np.random.seed(seed=29)
weights = (np.random.rand(*weight_dims)-0.5) / 2 # np.ones((weight_dims)) # 
biases = (np.random.rand(weight_dims[0])-0.5) # np.zeros((weight_dims[0])) # 

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights, biases=biases),
    layers.MonitorLayer(),
])

input0 = quartz.probe(loihi_model.layers[0].blocks[0])
trigger0 = quartz.probe(loihi_model.layers[0].blocks[-1])
trigger1 = quartz.probe(loihi_model.layers[1].blocks[0])
hidden0 = quartz.probe(loihi_model.layers[1].blocks[2])
hidden1 = quartz.probe(loihi_model.layers[1].blocks[3])
#hidden2 = quartz.probe(loihi_model.layers[1].blocks[4])
calc0 = quartz.probe(loihi_model.layers[1].blocks[1].neurons[0])
#sync0 = quartz.probe(loihi_model.layers[1].blocks[2].neurons[2])
#calc1 = quartz.probe(loihi_model.layers[1].blocks[4].neurons[0])
#sync1 = quartz.probe(loihi_model.layers[1].blocks[3].neurons[2])
#calc2 = quartz.probe(loihi_model.layers[1].blocks[4].neurons[0])
#sync2 = quartz.probe(loihi_model.layers[1].blocks[4].neurons[2])

values = np.random.rand(np.product(input_dims)) / 2 # np.zeros((np.product(input_dims)))
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)

weight_acc = 2**7
quantized_values = (values*t_max).round()/t_max
quantized_values = quantized_values.reshape(*input_dims)
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=kernel_size), nn.ReLU())
model[0].weight = torch.nn.Parameter(torch.tensor(quantized_weights))
model[0].bias = torch.nn.Parameter(torch.tensor(quantized_biases))
model_output = model(torch.tensor(quantized_values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()
model_output.flatten()

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
calc0.plot()

In [ ]:
trigger0.plot()

In [ ]:
trigger1.plot()

In [ ]:
loihi_model.layers[1].blocks[0].neurons[0].outgoing_synapses()

In [ ]:
loihi_model.layers[1].blocks[1].neurons[0].incoming_synapses()

In [ ]:
sync0.plot()

In [ ]:
hidden0.plot()

In [ ]:
loihi_model.layers[1].blocks

## Max Pooling layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

t_max = 2**8
input_dims = (1,10,10)
kernel_size = [2,2]

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.MaxPool2D(kernel_size=kernel_size),
    layers.MonitorLayer(),
])

pool0 = quartz.probe(loihi_model.layers[1].blocks[0])

np.random.seed(seed=45)
values = np.random.rand(np.product(input_dims))
values[values.shape[0]//2:] = 0
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)
quantized_values = (values*t_max).round()/t_max

model = nn.Sequential(nn.MaxPool2d(kernel_size=kernel_size, stride=kernel_size[0]), nn.ReLU())
model_output = model(torch.tensor(quantized_values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
values